In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt


In [ ]:
# setup train data

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)


In [ ]:
len(train_data), len(test_data)

In [ ]:
image, label = train_data[0]
image, label, image.shape 
#1， 28， 28 ->28x28pixel grayscale image

In [ ]:
train_data.classes, train_data.class_to_idx

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(image.squeeze())

In [ ]:
plt.imshow(image.squeeze(), cmap='gray')
plt.title(train_data.classes[label])

In [ ]:
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows,cols = 4,4
for i in range(1, rows*cols+1):
    rnd = torch.randint(0, len(train_data),size=[1]).item()
    img, label = train_data[rnd]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(train_data.classes[label])
    plt.axis(False)


In [ ]:
# Use batches

from torch.utils.data import DataLoader

BATCH_SIZE = 32
train_dataloader = DataLoader(train_data, BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, BATCH_SIZE, shuffle=True)

# 60000/32 = 1875
len(train_dataloader)

In [ ]:

train_batch, train_label_batch = next(iter(train_dataloader))
train_batch.shape,train_label_batch

In [ ]:
# get first item of this batch
img, label = train_batch[0],train_label_batch[0]
plt.imshow(img.squeeze(),cmap='gray')
plt.title(train_data.classes[label])

In [ ]:
flattern = nn.Flatten()

output = flattern(img)

img.shape, output.shape

In [ ]:
HIDDEN = 10
device ='cpu'

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(in_features=28*28, out_features=HIDDEN),
    nn.Linear(in_features=HIDDEN,out_features=10),
).to(device)

torch.manual_seed(42)

model(img)


In [ ]:
import requests
from pathlib import Path 

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

#for multi-class issue, use nn.CrossEntropyLoss()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [ ]:
from tqdm.auto import tqdm

torch.manual_seed(42)

epochs = 3

for epoch in tqdm(range(epochs)):
    print (f"E={epoch}")
    train_loss = 0
    for batch, (X, y ) in enumerate(train_dataloader):
        model.train()

        y_pred = model(X)
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y)
        train_loss += loss
        loss.backward()
        optimizer.step()

        if(batch % 400 ==0):
            print(f"{batch * BATCH_SIZE} / {len(train_dataloader.dataset)}")  #len=60000
    train_loss/=len(train_dataloader) # len=1875
    print(f"train_loss:{train_loss}")



In [ ]:
# # Import tqdm for progress bar
# from tqdm.auto import tqdm

# # Set the seed and start the timer
# torch.manual_seed(42)


# # Set the number of epochs (we'll keep this small for faster training times)
# epochs = 3

# # Create training and testing loop
# for epoch in tqdm(range(epochs)):
#     print(f"Epoch: {epoch}\n-------")
#     ### Training
#     train_loss = 0
#     # Add a loop to loop through training batches
#     for batch, (X, y) in enumerate(train_dataloader):
#         model.train() 
#         # 1. Forward pass
#         y_pred = model(X)

#         # 2. Calculate loss (per batch)
#         loss = loss_fn(y_pred, y)
#         train_loss += loss # accumulatively add up the loss per epoch 

#         # 3. Optimizer zero grad
#         optimizer.zero_grad()

#         # 4. Loss backward
#         loss.backward()

#         # 5. Optimizer step
#         optimizer.step()

#         # Print out how many samples have been seen
#         if batch % 400 == 0:
#             print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

#     # Divide total train loss by length of train dataloader (average loss per batch per epoch)
#     train_loss /= len(train_dataloader)
    
#     ### Testing
#     # Setup variables for accumulatively adding up loss and accuracy 
#     test_loss, test_acc = 0, 0 
#     model.eval()
#     with torch.inference_mode():
#         for X, y in test_dataloader:
#             # 1. Forward pass
#             test_pred = model(X)
           
#             # 2. Calculate loss (accumatively)
#             test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch

#             # 3. Calculate accuracy (preds need to be same as y_true)
#             test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
        
#         # Calculations on test metrics need to happen inside torch.inference_mode()
#         # Divide total test loss by length of test dataloader (per batch)
#         test_loss /= len(test_dataloader)

#         # Divide total accuracy by length of test dataloader (per batch)
#         test_acc /= len(test_dataloader)

#     ## Print out what's happening
#     print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")



In [ ]:
HIDDEN = 10
device ='cpu'

model2 = nn.Sequential(
    nn.Flatten(),
    nn.Linear(in_features=28*28, out_features=HIDDEN),
    nn.Linear(in_features=HIDDEN,out_features=10),
).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model2.parameters(), lr = 0.1)


def train_step(model: torch.nn.Module,
               data_loader,
               loss_fn: torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               ):
    train_loss = 0
    for batch, (X, y ) in enumerate(data_loader):
        model.train()

        y_pred = model(X)
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y)
        train_loss += loss
        loss.backward()
        optimizer.step()

        if(batch % 400 ==0):
            print(f"{batch * BATCH_SIZE} / {len(train_dataloader.dataset)}")  #len=60000
    train_loss/=len(train_dataloader) # len=1875
    print(f"train_loss:{train_loss}")





In [ ]:
torch.manual_seed(42)

epochs = 3

for epoch in range(epochs):
    train_step(model2, train_dataloader, loss_fn, optimizer)